In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy

In [ ]:
from tueplots import bundles



# Inspired by bundles.neurips2023(), but adapting font sizes for pt12 standard

settings_dict = {'text.usetex': True,
                 'font.family': 'serif',
                 'text.latex.preamble': '\\renewcommand{\\rmdefault}{ptm}\\renewcommand{\\sfdefault}{phv}',
                 'figure.figsize': (5.5, 3.399186938124422),
                 'figure.constrained_layout.use': True,
                 'figure.autolayout': False,
                 'savefig.bbox': 'tight',
                 'savefig.pad_inches': 0.015,
                 'font.size': 10,
                 'axes.labelsize': 10,
                 'legend.fontsize': 8,
                 'xtick.labelsize': 8,
                 'ytick.labelsize': 8,
                 'axes.titlesize': 10,
                 'figure.dpi': 300}


plt.rcParams.update(settings_dict)


# Can use colors from bundles.rgb.
#     tue_blue
#     tue_brown
#     tue_dark
#     tue_darkblue
#     tue_darkgreen
#     tue_gold
#     tue_gray
#     tue_green
#     tue_lightblue
#     tue_lightgold
#     tue_lightgreen
#     tue_lightorange
#     tue_mauve
#     tue_ocre
#     tue_orange
#     tue_red
#     tue_violet

In [ ]:
# # %matplotlib widget
# import matplotlib
# matplotlib.rcParams["figure.dpi"] = 300

In [ ]:
# The 'laplace-redux/results' folder has to be in the following specified folder:
RESULTS_DIRECTORY = 'results - amazon, camelyon17, SkinLesions - all conditions'


In [ ]:
# Due to file length restrictions, I renamed 
# camelyon17_wrn50_laplace_ef_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted_1254883.npy -> camelyon17_wrn50_laplace_ef_weight_inc_temp_last_layer_full_sf_df_dsf_1254883.npy
# camelyon17_RN50_laplace_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted_1254883.npy -> camelyon17_RN50_laplace_weight_inc_temp_last_layer_full_df_df_dsf_1254883.npy
# camelyon17_resnet50_laplace_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted_1254883.npy -> camelyon17_resnet50_laplace_weight_inc_temp_last_layer_full_sf_df_dsf_1254883.npy




In [ ]:
# ### TESTING: for some reason, there have been inftys produced in the nll calculation
# # -> I am not sure why: It is not due to errors in Cov-scaling, as there are also conditions without Cov-scaling that have them
# # Also, ECE calculation did work
# # It is probably due to log(0) values somewhere, but not sure how...
# # Probably due to very high overconfidence in the wrong class, resulting in -log(0) for the target class in NLL

# dirs = os.listdir(os.path.join(RESULTS_DIRECTORY, 'SkinLesions'))

# dirs = [d for d in dirs if 'skinlesions_wrn50' in d]

# for d in dirs:
#     dat = np.load(f'{RESULTS_DIRECTORY}/SkinLesions/{d}', allow_pickle=True)
#     print('##### ', d)
#     print(dat)

In [ ]:
def load_results(RESULTS_DIRECTORY, dataset_name, sub_name, model_ids):

    dat_list = []
    for i in model_ids:
        dat_list.append(np.load(f'{RESULTS_DIRECTORY}/{dataset_name}/{sub_name}_{i}.npy', allow_pickle=True))
    
    dat = []
    for i in range(len(dat_list[0])):
        dat_i = {}
        for k, v in dat_list[0][i].items():
            values_list = [model_seed_run[i][k] for model_seed_run in dat_list]
            dat_i[k] = np.mean(values_list)
            dat_i[k + "_se"] = np.std(values_list) / np.sqrt(len(values_list))
        dat.append(dat_i)

    return dat


In [ ]:
# automatically load all conditions

def get_all_conditions_model_id_names(RESULTS_DIRECTORY, DATASET_NAME):
    results_files = os.listdir(os.path.join(RESULTS_DIRECTORY, DATASET_NAME))
    conditions = list(set(["_".join(f.split("_")[:-1]) for f in results_files]))
    model_ids = []
    for condition in conditions:
        this_condition_files = [f for f in results_files if f[:len(condition)] == condition]
        this_condition_ids = list(set([f.split("_")[-1][:-4] for f in this_condition_files]))
        this_condition_ids.sort()
        model_ids.append(this_condition_ids)
    return conditions, model_ids


DATASET = "WILDS-amazon"
conditions_amazon, model_ids_amazon = get_all_conditions_model_id_names(RESULTS_DIRECTORY, DATASET)
# model_ids_amazon = [] # TODO remove, testing if results from laplace-redux are obtained when only using model 0
results_amazon = []
for condition, model_ids in zip(conditions_amazon, model_ids_amazon): 
    results_amazon.append(load_results(RESULTS_DIRECTORY, DATASET, condition, model_ids))


DATASET = "WILDS-camelyon17"
conditions_camelyon17, model_ids_camelyon17 = get_all_conditions_model_id_names(RESULTS_DIRECTORY, DATASET)
# model_ids_camelyon17 = [5] # TODO remove, testing if results from laplace-redux are obtained when only using model 0
results_camelyon17 = []
for condition, model_ids in zip(conditions_camelyon17, model_ids_camelyon17): 
    results_camelyon17.append(load_results(RESULTS_DIRECTORY, DATASET, condition, model_ids))


DATASET = "SkinLesions"
conditions_SkinLesions, model_ids_SkinLesions = get_all_conditions_model_id_names(RESULTS_DIRECTORY, DATASET)
results_SkinLesions = []
for condition, model_ids in zip(conditions_SkinLesions, model_ids_SkinLesions): 
    results_SkinLesions.append(load_results(RESULTS_DIRECTORY, DATASET, condition, model_ids))


# load all results for each condition into a list.
# make a list of all the condition names

In [ ]:
# results_camelyon17

In [ ]:
condition_translation_dict = {
    'laplace_ef_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "LLLA(EF)+CVS",
    'laplace_last_layer_full': "LLLA", 
    'laplace_last_layer_full_diagaddfitted': "LLLA+CVS - only diag add",
    'laplace_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "LLLA+CVS",
    'temp': "TS (pycalib)",
    'laplace_weight_inc_temp_last_layer_full': "LLLA+WITS",
    'map_weight_inc_temp': "TS (WITS)",
    'laplace_ef_weight_inc_temp_last_layer_full': "LLLA(EF)+WITS",
    'laplace_ef_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "LLLA(EF)+WITS+CVS",
    'map': "MAP",
    'laplace_last_layer_full_scalingfittted': "LLLA+CVS - only scaling",
    'laplace_ef_last_layer_full': "LLLA(EF)",
    'laplace_last_layer_full_diagscalingfitted': "LLLA+CVS - only diag scaling",
    'laplace_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "LLLA+WITS+CVS",
    'laplace_OODValSet_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "fitted on OOD: LLLA+WITS+CVS",
    'laplace_OODValSet_weight_inc_temp_last_layer_full': "fitted on OOD: LLLA+WITS",
    'laplace_OODValSet_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "fitted on OOD: LLLA+CVS",
    'laplace_ef_OODValSet_weight_inc_temp_last_layer_full': "fitted on OOD: LLLA(EF)+WITS",
    'laplace_ef_OODValSet_last_layer_full': "fitted on OOD: LLLA(EF)",
    'laplace_ef_OODValSet_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "fitted on OOD: LLLA(EF)+CVS",
    'laplace_ef_OODValSet_weight_inc_temp_last_layer_full_scalingfittted_diagaddfitted_diagscalingfitted': "fitted on OOD: LLLA(EF)+WITS+CVS",
    'map_OODValSet_weight_inc_temp': "fitted on OOD: TS (WITS)",
}


In [ ]:
condition_translation_dict_network_ablation = {}
for k, v in condition_translation_dict.items():
    if 'camelyon17_wrn50_' + k in conditions_camelyon17:
        condition_translation_dict_network_ablation['camelyon17_wrn50_' + k] = 'WRN50 ' + v
    if 'camelyon17_resnet50_' + k in conditions_camelyon17:
        condition_translation_dict_network_ablation['camelyon17_resnet50_' + k] = 'RN50 ' + v

    if 'skinlesions_wrn50_' + k in conditions_SkinLesions:
        condition_translation_dict_network_ablation['skinlesions_wrn50_' + k] = 'WRN50 ' + v


In [ ]:
condition_translation_dict_network_ablation.update({
    'camelyon17_wrn50_laplace_ef_weight_inc_temp_last_layer_full_sf_df_dsf': 'WRN50 LLLA(EF)+WITS+CVS',
    'camelyon17_wrn50_laplace_weight_inc_temp_last_layer_full_df_df_dsf': 'WRN50 LLLA+WITS+CVS',
    'camelyon17_resnet50_laplace_weight_inc_temp_last_layer_full_sf_df_dsf': 'RN50 LLLA+WITS+CVS',
})

In [ ]:
condition_translation_dict.update(condition_translation_dict_network_ablation)

In [ ]:
assert np.all([c in condition_translation_dict.keys() for c in conditions_camelyon17])


In [ ]:
assert np.all([c in condition_translation_dict.keys() for c in conditions_amazon])


In [ ]:
assert np.all([c in condition_translation_dict.keys() for c in conditions_SkinLesions])


# Nice combined plot

In [ ]:
conditions_camelyon17 = [condition_translation_dict[c] for c in conditions_camelyon17]
conditions_amazon = [condition_translation_dict[c] for c in conditions_amazon]
conditions_SkinLesions = [condition_translation_dict[c] for c in conditions_SkinLesions]

In [ ]:
total_amount_of_conditions = len(set(conditions_amazon + conditions_camelyon17 + conditions_SkinLesions))

In [ ]:
total_amount_of_conditions

In [ ]:
all_unique_conditions = set(conditions_amazon + conditions_camelyon17 + conditions_SkinLesions)

all_unique_conditions = [c for c in all_unique_conditions if c not in condition_translation_dict_network_ablation.values()]

all_network_ablation_conditions = list(condition_translation_dict_network_ablation.values())

fitted_on_OOD_conditions = [c for c in all_unique_conditions if 'fitted on OOD' in c]
standard_conditions = [c for c in all_unique_conditions if c not in fitted_on_OOD_conditions]

fitted_on_OOD_ef_conditions = [c for c in fitted_on_OOD_conditions if 'EF' in c]
fitted_on_OOD_conditions = [c for c in fitted_on_OOD_conditions if c not in fitted_on_OOD_ef_conditions]
ef_conditions = [c for c in standard_conditions if "EF" in c]
standard_conditions = [c for c in standard_conditions if c not in ef_conditions]
scaling_ablation_conditions = [c for c in standard_conditions if "only" in c]
standard_conditions = [c for c in standard_conditions if c not in scaling_ablation_conditions]


In [ ]:
all_unique_conditions = list(all_unique_conditions)
all_unique_conditions.sort()
standard_conditions = list(standard_conditions)
standard_conditions.sort()

fitted_on_OOD_conditions = list(fitted_on_OOD_conditions)
fitted_on_OOD_conditions.sort()
fitted_on_OOD_ef_conditions = list(fitted_on_OOD_ef_conditions)
fitted_on_OOD_ef_conditions.sort()
ef_conditions = list(ef_conditions)
ef_conditions.sort()
scaling_ablation_conditions = list(scaling_ablation_conditions)
scaling_ablation_conditions.sort()


In [ ]:
all_unique_conditions

In [ ]:
standard_conditions

In [ ]:
fitted_on_OOD_conditions

In [ ]:
fitted_on_OOD_ef_conditions

In [ ]:
ef_conditions

In [ ]:
scaling_ablation_conditions

In [ ]:
all_network_ablation_conditions

# Table of accuracies

In [ ]:
print("\\begin{table}\n\\begin{center}\n\\begin{tabular}{l|cc}")
print("    Dataset / Model & ID & OOD \\\\")
for conditions, results, dataset in zip([conditions_amazon, conditions_camelyon17, conditions_SkinLesions],
                                        [results_amazon, results_camelyon17, results_SkinLesions],
                                        ['WILDS-Amazon', 'WILDS-Camelyon17', 'SkinLesions']):

    print("    \\hline")
    print("    \\textbf{" + dataset + "} & & \\\\")

    for condition, result in zip(conditions, results):
        ret = "        \\hspace{5mm} " + condition + " & "
        ret += " & ".join([f"{i['acc'] * 100:.1f}" for i in result])
        ret += " \\\\"
        print(ret)

print("\\end{tabular}\n\\end{center}\n\\caption{[TODO]}\\label{[TODO]}\n\\end{table}")

# Plotting

In [ ]:
### Color palette

palette = plt.get_cmap('Set1')
condition_to_color = {c: palette(i / len(standard_conditions)) for i, c in enumerate(standard_conditions)}
palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(fitted_on_OOD_conditions)) for i, c in enumerate(fitted_on_OOD_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(ef_conditions)) for i, c in enumerate(ef_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(fitted_on_OOD_ef_conditions)) for i, c in enumerate(fitted_on_OOD_ef_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(scaling_ablation_conditions)) for i, c in enumerate(scaling_ablation_conditions)})

condition_to_color["MAP"] = "darkblue"

In [ ]:


def combined_plot(conditions_amazon, results_amazon, conditions_camelyon17, results_camelyon17, conditions_SkinLesions, results_SkinLesions, legend_order_permutation=None, fmtlist = None):
    fig, ax = plt.subplots(2, 3)
    # fig.set_size_inches([12, 8])

    if not fmtlist:
        fmtlist = [["o-" for _ in range(len(conditions_amazon))] for _ in range(6)]

    for condition, results, fmt in zip(conditions_amazon, results_amazon, fmtlist[0]):
        datasets = range(len(results))
        # ax[0][0].plot(datasets, [i['ece'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[0][0].errorbar(datasets, [i['ece'] for i in results], yerr = [i['ece_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[0][0].set_ylabel("ECE")
    ax[0][0].set_xticks([0,1])
    ax[0][0].set_xticklabels(["ID", "OOD"])
    # ax[0][0].legend() # TODO testing


    for condition, results, fmt in zip(conditions_amazon, results_amazon, fmtlist[1]):
        datasets = range(len(results))
        # ax[1][0].plot(datasets, [i['nll'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[1][0].errorbar(datasets, [i['nll'] for i in results], yerr = [i['nll_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[1][0].set_xticks([0,1])
    ax[1][0].set_xticklabels(["ID", "OOD"])
    ax[1][0].set_ylabel("NLL")
    # ax[1][0].set_xlabel("WILDS-amazon\n(Text classification)")
    ax[1][0].set_xlabel("WILDS-Amazon")
    # ax[1][0].legend() # TODO testing


    for condition, results, fmt in zip(conditions_camelyon17, results_camelyon17, fmtlist[2]):
        datasets = range(len(results))
        # ax[0][1].plot(datasets, [i['ece'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[0][1].errorbar(datasets, [i['ece'] for i in results], yerr = [i['ece_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[0][1].set_xticks([0,1])
    ax[0][1].set_xticklabels(["ID", "OOD"])
    # ax[0][1].legend() # TODO testing


    for condition, results, fmt in zip(conditions_camelyon17, results_camelyon17, fmtlist[3]):
        datasets = range(len(results))
        # ax[1][1].plot(datasets, [i['nll'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[1][1].errorbar(datasets, [i['nll'] for i in results], yerr = [i['nll_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    # ax[1][1].set_xlabel("WILDS-camelyon17\n(Histopathology tumor detection)")
    ax[1][1].set_xlabel("WILDS-Camelyon17")
    ax[1][1].set_xticks([0,1])
    ax[1][1].set_xticklabels(["ID", "OOD"])
    # ax[1][1].legend() # TODO testing


    for condition, results, fmt in zip(conditions_SkinLesions, results_SkinLesions, fmtlist[4]):
        datasets = range(len(results))
        # ax[0][2].plot(datasets, [i['ece'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[0][2].errorbar(datasets, [i['ece'] for i in results], yerr = [i['ece_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[0][2].set_xticks([0,1])
    ax[0][2].set_xticklabels(["ID", "OOD"])
    # ax[0][2].legend() # TODO testing


    for condition, results, fmt in zip(conditions_SkinLesions, results_SkinLesions, fmtlist[5]):
        datasets = range(len(results))
        # ax[1][2].plot(datasets, [i['nll'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[1][2].errorbar(datasets, [i['nll'] for i in results], yerr = [i['nll_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    # ax[1][2].set_xlabel("SkinLesions\n(Dermatoscopic tumor detection)")
    ax[1][2].set_xlabel("SkinLesions")
    ax[1][2].set_xticks([0,1])
    ax[1][2].set_xticklabels(["ID", "OOD"])
    # ax[1][2].legend() # TODO testing


    handles, labels = ax[1][2].get_legend_handles_labels()
    if legend_order_permutation:
        handles, labels = [handles[idx] for idx in legend_order_permutation],[labels[idx] for idx in legend_order_permutation]

    # plt.figlegend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.55))
    ax[0][2].legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))
    # plt.tight_layout()
    # plt.show()



In [ ]:
def filter_conditions(conditions, results, wanted_conditions_list):
    ret_conditions, ret_results = [], []

    for c, r in zip(conditions, results): 
        if c in wanted_conditions_list:
            ret_conditions.append(c)
            ret_results.append(r)

    firstprint = True
    for c in wanted_conditions_list:
        if c not in conditions:
            if firstprint:
                print("#####")
                firstprint=False

            print(f"'{c}' not found in conditions")

    return ret_conditions, ret_results

In [ ]:
wanted_conditions = all_unique_conditions
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

conditions_amazon_filtered, results_amazon_filtered = zip(*sorted(zip(conditions_amazon_filtered, results_amazon_filtered)))
conditions_camelyon17_filtered, results_camelyon17_filtered = zip(*sorted(zip(conditions_camelyon17_filtered, results_camelyon17_filtered)))
conditions_SkinLesions_filtered, results_SkinLesions_filtered = zip(*sorted(zip(conditions_SkinLesions_filtered, results_SkinLesions_filtered)))

combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered)
plt.show()

In [ ]:
conditions_amazon_filtered

In [ ]:
# TODO
# Concise plots:
# Baseline (MAP, LLLA, TS(pycalib)) (+ TS(mine?))
wanted_conditions = ["MAP", "LLLA", "TS (pycalib)", "TS (WITS)"]
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

conditions_amazon_filtered, results_amazon_filtered = zip(*sorted(zip(conditions_amazon_filtered, results_amazon_filtered)))
conditions_camelyon17_filtered, results_camelyon17_filtered = zip(*sorted(zip(conditions_camelyon17_filtered, results_camelyon17_filtered)))
conditions_SkinLesions_filtered, results_SkinLesions_filtered = zip(*sorted(zip(conditions_SkinLesions_filtered, results_SkinLesions_filtered)))

conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered = list(conditions_amazon_filtered), list(results_amazon_filtered), list(conditions_camelyon17_filtered), list(results_camelyon17_filtered), list(conditions_SkinLesions_filtered), list(results_SkinLesions_filtered)
conditions_amazon_filtered[0], conditions_amazon_filtered[1] = conditions_amazon_filtered[1], conditions_amazon_filtered[0]
results_amazon_filtered[0], results_amazon_filtered[1] = results_amazon_filtered[1], results_amazon_filtered[0]
conditions_camelyon17_filtered[0], conditions_camelyon17_filtered[1] = conditions_camelyon17_filtered[1], conditions_camelyon17_filtered[0]
results_camelyon17_filtered[0], results_camelyon17_filtered[1] = results_camelyon17_filtered[1], results_camelyon17_filtered[0]
conditions_SkinLesions_filtered[0], conditions_SkinLesions_filtered[1] = conditions_SkinLesions_filtered[1], conditions_SkinLesions_filtered[0]
results_SkinLesions_filtered[0], results_SkinLesions_filtered[1] = results_SkinLesions_filtered[1], results_SkinLesions_filtered[0]

condition_to_color.update({'MAP': bundles.rgb.tue_red,
                          'TS (pycalib)': bundles.rgb.tue_lightorange,
                          "TS (WITS)": bundles.rgb.tue_orange,
                          'LLLA': bundles.rgb.tue_violet,
                          'LLLA+CVS': bundles.rgb.tue_lightblue,
                          'LLLA+WITS': bundles.rgb.tue_blue,
                          'LLLA+WITS+CVS': bundles.rgb.tue_darkblue,
                          "fitted on OOD: TS (WITS)": bundles.rgb.tue_gray,
                          'fitted on OOD: LLLA+CVS': bundles.rgb.tue_lightgreen,
                          'fitted on OOD: LLLA+WITS': bundles.rgb.tue_green,
                          'fitted on OOD: LLLA+WITS+CVS': bundles.rgb.tue_darkgreen
})




fmtlist = [["o-", "o--", "o-", "o--"],
           ["o-", "o--", "o-", "o--"],
           ["o-", "o:", "o--", "o-"],
           ["o-", "o:", "o--", "o-"],
           ["o-", "o-", "o-", "o-"],
           ["o-", "o-", "o-", "o-"]]
combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, fmtlist=fmtlist, legend_order_permutation=[0, 3, 2, 1])
if not os.path.exists('img/Results/RealWorld_ECE_NLL/'):
    os.makedirs('img/Results/RealWorld_ECE_NLL/')
plt.savefig('img/Results/RealWorld_ECE_NLL/MAP_TS_LLLA_ECE_NLL.pdf')
plt.show()


In [ ]:
# WITS / Laplace with Cov-scaling

wanted_conditions = deepcopy(standard_conditions)
wanted_conditions.remove("TS (pycalib)")
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

conditions_amazon_filtered, results_amazon_filtered = zip(*sorted(zip(conditions_amazon_filtered, results_amazon_filtered)))
conditions_camelyon17_filtered, results_camelyon17_filtered = zip(*sorted(zip(conditions_camelyon17_filtered, results_camelyon17_filtered)))
conditions_SkinLesions_filtered, results_SkinLesions_filtered = zip(*sorted(zip(conditions_SkinLesions_filtered, results_SkinLesions_filtered)))

conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered = list(conditions_amazon_filtered), list(results_amazon_filtered), list(conditions_camelyon17_filtered), list(results_camelyon17_filtered), list(conditions_SkinLesions_filtered), list(results_SkinLesions_filtered)
conditions_amazon_filtered[0], conditions_amazon_filtered[4] = conditions_amazon_filtered[4], conditions_amazon_filtered[0]
results_amazon_filtered[0], results_amazon_filtered[4] = results_amazon_filtered[4], results_amazon_filtered[0]
conditions_camelyon17_filtered[0], conditions_camelyon17_filtered[4] = conditions_camelyon17_filtered[4], conditions_camelyon17_filtered[0]
results_camelyon17_filtered[0], results_camelyon17_filtered[4] = results_camelyon17_filtered[4], results_camelyon17_filtered[0]
conditions_SkinLesions_filtered[0], conditions_SkinLesions_filtered[4] = conditions_SkinLesions_filtered[4], conditions_SkinLesions_filtered[0]
results_SkinLesions_filtered[0], results_SkinLesions_filtered[4] = results_SkinLesions_filtered[4], results_SkinLesions_filtered[0]

fmtlist = [["o-", "o-", "o-", "o--", "o--", "o--"],
           ["o-", "o-", "o-", "o--", "o--", "o--"],
           ["o-", "o-", "o--", "o:", "o--", "o:"],
           ["o-", "o-", "o--", "o:", "o--", "o:"],
           ["o-", "o-", "o-", "o-", "o--", "o-"],
           ["o-", "o-", "o-", "o-", "o-", "o-"]]



combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, fmtlist=fmtlist, legend_order_permutation=[0, 5, 3, 2, 4, 1])
if not os.path.exists('img/Results/RealWorld_ECE_NLL/'):
    os.makedirs('img/Results/RealWorld_ECE_NLL/')
plt.savefig('img/Results/RealWorld_ECE_NLL/WITS_CovScaling_NLL_ECE.pdf')
plt.show()


In [ ]:
# Examine EF

wanted_conditions = ["MAP", "LLLA(EF)+WITS+CVS", "LLLA(EF)+WITS", "LLLA(EF)", "LLLA(EF)+CVS"]
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

conditions_amazon_filtered, results_amazon_filtered = zip(*sorted(zip(conditions_amazon_filtered, results_amazon_filtered)))
conditions_camelyon17_filtered, results_camelyon17_filtered = zip(*sorted(zip(conditions_camelyon17_filtered, results_camelyon17_filtered)))
conditions_SkinLesions_filtered, results_SkinLesions_filtered = zip(*sorted(zip(conditions_SkinLesions_filtered, results_SkinLesions_filtered)))

conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered = list(conditions_amazon_filtered), list(results_amazon_filtered), list(conditions_camelyon17_filtered), list(results_camelyon17_filtered), list(conditions_SkinLesions_filtered), list(results_SkinLesions_filtered)
conditions_amazon_filtered[0], conditions_amazon_filtered[4] = conditions_amazon_filtered[4], conditions_amazon_filtered[0]
results_amazon_filtered[0], results_amazon_filtered[4] = results_amazon_filtered[4], results_amazon_filtered[0]
conditions_camelyon17_filtered[0], conditions_camelyon17_filtered[4] = conditions_camelyon17_filtered[4], conditions_camelyon17_filtered[0]
results_camelyon17_filtered[0], results_camelyon17_filtered[4] = results_camelyon17_filtered[4], results_camelyon17_filtered[0]
conditions_SkinLesions_filtered[0], conditions_SkinLesions_filtered[4] = conditions_SkinLesions_filtered[4], conditions_SkinLesions_filtered[0]
results_SkinLesions_filtered[0], results_SkinLesions_filtered[4] = results_SkinLesions_filtered[4], results_SkinLesions_filtered[0]

condition_to_color.update({'MAP': bundles.rgb.tue_red,
                          'TS (pycalib)': bundles.rgb.tue_lightorange,
                          "TS (WITS)": bundles.rgb.tue_orange,
                          'LLLA(EF)': bundles.rgb.tue_violet,
                          'LLLA(EF)+CVS': bundles.rgb.tue_lightblue,
                          'LLLA(EF)+WITS': bundles.rgb.tue_blue,
                          'LLLA(EF)+WITS+CVS': bundles.rgb.tue_darkblue,
                          "fitted on OOD: TS (WITS)": bundles.rgb.tue_gray,
                          'fitted on OOD: LLLA+CVS': bundles.rgb.tue_lightgreen,
                          'fitted on OOD: LLLA+WITS': bundles.rgb.tue_green,
                          'fitted on OOD: LLLA+WITS+CVS': bundles.rgb.tue_darkgreen
})


fmtlist = [["o-", "o-", "o--", "o--", "o--"],
           ["o-", "o-", "o--", "o--", "o--"],
           ["o-", "o--", "o:", "o--", "o:"],
           ["o-", "o--", "o:", "o--", "o:"],
           ["o-", "o-", "o-", "o-", "o--"],
           ["o-", "o-", "o-", "o-", "o--"]]

combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, fmtlist=fmtlist, legend_order_permutation=[0, 4, 2, 1, 3])
if not os.path.exists('img/Results/RealWorld_ECE_NLL/'):
    os.makedirs('img/Results/RealWorld_ECE_NLL/')
plt.savefig('img/Results/RealWorld_ECE_NLL/WITS_CovScaling_NLL_ECE_EF_Ablation.pdf')
plt.show()


In [ ]:
# # Directly comparing WITS, Laplace Cov-scaling, both

# wanted_conditions = ["MAP", "LLLA+WITS", "LLLA+CVS", "LLLA+WITS+CVS"]
# conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
# conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)

# combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, legend_order_permutation=[3, 0, 1, 2])


In [ ]:

# TODO make this plot with Amazon-OOD as soon as I have this data

def combined_plot_OnlyOOD(conditions_camelyon17, results_camelyon17, legend_order_permutation=None):
    fig, ax = plt.subplots(2, 1)
    # fig.set_size_inches([12, 8])


    for condition, results in zip(conditions_camelyon17, results_camelyon17):
        datasets = range(len(results))
        # ax[0][0].plot(datasets, [i['ece'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[0].errorbar(datasets, [i['ece'] for i in results], yerr = [i['ece_se'] for i in results], fmt="o-", label=condition, color=condition_to_color[condition])
    ax[0].set_xticks([0,1])
    ax[0].set_ylabel("ECE")
    ax[0].set_xticklabels(["ID", "OOD"])
    # ax[0][0].legend() # TODO testing


    for condition, results in zip(conditions_camelyon17, results_camelyon17):
        datasets = range(len(results))
        # ax[1].plot(datasets, [i['nll'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[1].errorbar(datasets, [i['nll'] for i in results], yerr = [i['nll_se'] for i in results], fmt="o-", label=condition, color=condition_to_color[condition])
    # ax[1].set_xlabel("WILDS-camelyon17\n(Histopathology tumor detection)")
    ax[1].set_xlabel("WILDS-Camelyon17")
    ax[1].set_xticks([0,1])
    ax[1].set_ylabel("NLL")
    ax[1].set_xticklabels(["ID", "OOD"])
    # ax[1].legend() # TODO testing

    handles, labels = ax[1].get_legend_handles_labels()
    if legend_order_permutation:
        handles, labels = [handles[idx] for idx in legend_order_permutation],[labels[idx] for idx in legend_order_permutation]

    # plt.figlegend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.55))
    ax[0].legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))
    # plt.tight_layout()
    # plt.show()



In [ ]:


def combined_plot_OnlyOODTwo(conditions_amazon, results_amazon, conditions_camelyon17, results_camelyon17, conditions_SkinLesions, results_SkinLesions, legend_order_permutation=None, fmtlist = None):
    fig, ax = plt.subplots(2, 2)
    # fig.set_size_inches([12, 8])

    if not fmtlist:
        fmtlist = [["o-" for _ in range(len(conditions_amazon))] for _ in range(6)]

    for condition, results, fmt in zip(conditions_amazon, results_amazon, fmtlist[0]):
        datasets = range(len(results))
        # ax[0][0].plot(datasets, [i['ece'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[0][0].errorbar(datasets, [i['ece'] for i in results], yerr = [i['ece_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[0][0].set_ylabel("ECE")
    ax[0][0].set_xticks([0,1])
    ax[0][0].set_xticklabels(["ID", "OOD"])
    # ax[0][0].legend() # TODO testing


    for condition, results, fmt in zip(conditions_amazon, results_amazon, fmtlist[1]):
        datasets = range(len(results))
        # ax[1][0].plot(datasets, [i['nll'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[1][0].errorbar(datasets, [i['nll'] for i in results], yerr = [i['nll_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[1][0].set_xticks([0,1])
    ax[1][0].set_xticklabels(["ID", "OOD"])
    ax[1][0].set_ylabel("NLL")
    # ax[1][0].set_xlabel("WILDS-amazon\n(Text classification)")
    ax[1][0].set_xlabel("WILDS-Amazon")
    # ax[1][0].legend() # TODO testing


    for condition, results, fmt in zip(conditions_camelyon17, results_camelyon17, fmtlist[2]):
        datasets = range(len(results))
        # ax[0][1].plot(datasets, [i['ece'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[0][1].errorbar(datasets, [i['ece'] for i in results], yerr = [i['ece_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    ax[0][1].set_xticks([0,1])
    ax[0][1].set_xticklabels(["ID", "OOD"])
    # ax[0][1].legend() # TODO testing


    for condition, results, fmt in zip(conditions_camelyon17, results_camelyon17, fmtlist[3]):
        datasets = range(len(results))
        # ax[1][1].plot(datasets, [i['nll'] for i in results], "o-", label=condition, color=condition_to_color[condition])
        ax[1][1].errorbar(datasets, [i['nll'] for i in results], yerr = [i['nll_se'] for i in results], fmt=fmt, label=condition, color=condition_to_color[condition])
    # ax[1][1].set_xlabel("WILDS-camelyon17\n(Histopathology tumor detection)")
    ax[1][1].set_xlabel("WILDS-Camelyon17")
    ax[1][1].set_xticks([0,1])
    ax[1][1].set_xticklabels(["ID", "OOD"])
    # ax[1][1].legend() # TODO testing

    handles, labels = ax[0][1].get_legend_handles_labels()
    if legend_order_permutation:
        handles, labels = [handles[idx] for idx in legend_order_permutation],[labels[idx] for idx in legend_order_permutation]

    # plt.figlegend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.55))
    ax[0][1].legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))
    # plt.tight_layout()
    # plt.show()



In [ ]:
# fitting on OODVal

wanted_conditions = standard_conditions + fitted_on_OOD_conditions
wanted_conditions = deepcopy(wanted_conditions)
wanted_conditions.remove("TS (pycalib)")
wanted_conditions.remove("LLLA")
wanted_conditions.remove("TS (WITS)")
wanted_conditions.remove("LLLA+CVS")
wanted_conditions.remove("LLLA+WITS")

# conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = [], []

conditions_amazon_filtered, results_amazon_filtered = zip(*sorted(zip(conditions_amazon_filtered, results_amazon_filtered)))
conditions_camelyon17_filtered, results_camelyon17_filtered = zip(*sorted(zip(conditions_camelyon17_filtered, results_camelyon17_filtered)))
# conditions_SkinLesions_filtered, results_SkinLesions_filtered = zip(*sorted(zip(conditions_SkinLesions_filtered, results_SkinLesions_filtered)))

fmtlist = [["o-", "o-", "o-", "o--", "o-", "o:"],
           ["o-", "o-", "o-", "o-", "o--", "o:"],
           ["o-", "o-", "o-", "o-", "o-", "o-"],
           ["o-", "o-", "o-", "o-", "o--", "o:"]]

combined_plot_OnlyOODTwo(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, fmtlist=fmtlist, legend_order_permutation=[1, 0, 5, 3, 2, 4])
if not os.path.exists('img/Results/RealWorld_ECE_NLL/'):
    os.makedirs('img/Results/RealWorld_ECE_NLL/')
plt.savefig('img/Results/RealWorld_ECE_NLL/OODVal_NLL_ECE_WithAmazon.pdf')
plt.show()


In [ ]:
# # fitting on OODVal

# wanted_conditions = standard_conditions + fitted_on_OOD_conditions
# wanted_conditions = deepcopy(wanted_conditions)
# wanted_conditions.remove("TS (pycalib)")
# wanted_conditions.remove("LLLA")
# wanted_conditions.remove("TS (WITS)")
# wanted_conditions.remove("LLLA+CVS")
# wanted_conditions.remove("LLLA+WITS")

# # conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
# conditions_amazon_filtered, results_amazon_filtered = [], []
# conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
# conditions_SkinLesions_filtered, results_SkinLesions_filtered = [], []

# combined_plot_OnlyOOD(conditions_camelyon17_filtered, results_camelyon17_filtered, legend_order_permutation=[1, 2, 5, 4, 3, 0])
# if not os.path.exists('img/Results/RealWorld_ECE_NLL/'):
#     os.makedirs('img/Results/RealWorld_ECE_NLL/')
# plt.savefig('img/Results/RealWorld_ECE_NLL/OODVal_NLL_ECE.pdf')
# plt.show()


# Ablation on scaling parameters

In [ ]:
wanted_conditions = deepcopy(scaling_ablation_conditions)
wanted_conditions += ["LLLA", 'LLLA+CVS']
# wanted_conditions.remove("TS (pycalib)")
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

conditions_amazon_filtered, results_amazon_filtered = zip(*sorted(zip(conditions_amazon_filtered, results_amazon_filtered)))
conditions_camelyon17_filtered, results_camelyon17_filtered = zip(*sorted(zip(conditions_camelyon17_filtered, results_camelyon17_filtered)))
conditions_SkinLesions_filtered, results_SkinLesions_filtered = zip(*sorted(zip(conditions_SkinLesions_filtered, results_SkinLesions_filtered)))

fmtlist = [["o-", "o-", "o:", "o:", "o-"],
           ["o-", "o-", "o:", "o:", "o-"],
           ["o-", "o-", "o:", "o-", "o:"],
           ["o-", "o-", "o:", "o-", "o:"],
           ["o-", "o-", "o-", "o-", "o-"],
           ["o-", "o-", "o-", "o-", "o--"]]
           
combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, fmtlist=fmtlist)
if not os.path.exists('img/Results/RealWorld_ECE_NLL/'):
    os.makedirs('img/Results/RealWorld_ECE_NLL/')
plt.savefig('img/Results/RealWorld_ECE_NLL/CVS_Parameter_Ablation.pdf')
plt.show()


# Network Ablation

In [ ]:
# all_network_ablation_conditions

In [ ]:
wrn50_all_unique_conditions = [c for c in all_network_ablation_conditions if c[:5] == 'WRN50']


wrn50_fitted_on_OOD_conditions = [c for c in wrn50_all_unique_conditions if 'fitted on OOD' in c]
wrn50_standard_conditions = [c for c in wrn50_all_unique_conditions if c not in wrn50_fitted_on_OOD_conditions]

wrn50_fitted_on_OOD_ef_conditions = [c for c in wrn50_fitted_on_OOD_conditions if 'EF' in c]
wrn50_fitted_on_OOD_conditions = [c for c in wrn50_fitted_on_OOD_conditions if c not in wrn50_fitted_on_OOD_ef_conditions]
wrn50_ef_conditions = [c for c in wrn50_standard_conditions if "EF" in c]
wrn50_standard_conditions = [c for c in wrn50_standard_conditions if c not in wrn50_ef_conditions]
wrn50_scaling_ablation_conditions = [c for c in wrn50_standard_conditions if "only" in c]
wrn50_standard_conditions = [c for c in wrn50_standard_conditions if c not in wrn50_scaling_ablation_conditions]

wrn50_all_unique_conditions = list(wrn50_all_unique_conditions)
wrn50_all_unique_conditions.sort()
wrn50_standard_conditions = list(wrn50_standard_conditions)
wrn50_standard_conditions.sort()

wrn50_fitted_on_OOD_conditions = list(wrn50_fitted_on_OOD_conditions)
wrn50_fitted_on_OOD_conditions.sort()
wrn50_fitted_on_OOD_ef_conditions = list(wrn50_fitted_on_OOD_ef_conditions)
wrn50_fitted_on_OOD_ef_conditions.sort()
wrn50_ef_conditions = list(wrn50_ef_conditions)
wrn50_ef_conditions.sort()
wrn50_scaling_ablation_conditions = list(wrn50_scaling_ablation_conditions)
wrn50_scaling_ablation_conditions.sort()


In [ ]:
### Color palette

palette = plt.get_cmap('Set1')
condition_to_color.update({c: palette(i / len(wrn50_standard_conditions)) for i, c in enumerate(wrn50_standard_conditions)})
palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(wrn50_fitted_on_OOD_conditions)) for i, c in enumerate(wrn50_fitted_on_OOD_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(wrn50_ef_conditions)) for i, c in enumerate(wrn50_ef_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(wrn50_fitted_on_OOD_ef_conditions)) for i, c in enumerate(wrn50_fitted_on_OOD_ef_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(wrn50_scaling_ablation_conditions)) for i, c in enumerate(wrn50_scaling_ablation_conditions)})

condition_to_color["MAP"] = "darkblue"

In [ ]:
wanted_conditions = wrn50_all_unique_conditions
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered)

In [ ]:
# TODO
# Concise plots:
# Baseline (MAP, LLLA, TS(pycalib)) (+ TS(mine?))
wanted_conditions = ["MAP", "LLLA", "TS (pycalib)", "TS (WITS)"]
wanted_conditions = ['WRN50 ' + c for c in wanted_conditions]
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, legend_order_permutation=[2, 1, 0, 3])


In [ ]:
# WITS / Laplace with Cov-scaling

wanted_conditions = deepcopy(wrn50_standard_conditions)
wanted_conditions.remove("WRN50 TS (pycalib)")
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)


combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, legend_order_permutation=[3, 1, 0, 5, 2, 4])


### Ablation Resnet50

In [ ]:
rn50_all_unique_conditions = [c for c in all_network_ablation_conditions if c[:4] == 'RN50']


rn50_fitted_on_OOD_conditions = [c for c in rn50_all_unique_conditions if 'fitted on OOD' in c]
rn50_standard_conditions = [c for c in rn50_all_unique_conditions if c not in rn50_fitted_on_OOD_conditions]

rn50_fitted_on_OOD_ef_conditions = [c for c in rn50_fitted_on_OOD_conditions if 'EF' in c]
rn50_fitted_on_OOD_conditions = [c for c in rn50_fitted_on_OOD_conditions if c not in rn50_fitted_on_OOD_ef_conditions]
rn50_ef_conditions = [c for c in rn50_standard_conditions if "EF" in c]
rn50_standard_conditions = [c for c in rn50_standard_conditions if c not in rn50_ef_conditions]
rn50_scaling_ablation_conditions = [c for c in rn50_standard_conditions if "only" in c]
rn50_standard_conditions = [c for c in rn50_standard_conditions if c not in rn50_scaling_ablation_conditions]

rn50_all_unique_conditions = list(rn50_all_unique_conditions)
rn50_all_unique_conditions.sort()
rn50_standard_conditions = list(rn50_standard_conditions)
rn50_standard_conditions.sort()

rn50_fitted_on_OOD_conditions = list(rn50_fitted_on_OOD_conditions)
rn50_fitted_on_OOD_conditions.sort()
rn50_fitted_on_OOD_ef_conditions = list(rn50_fitted_on_OOD_ef_conditions)
rn50_fitted_on_OOD_ef_conditions.sort()
rn50_ef_conditions = list(rn50_ef_conditions)
rn50_ef_conditions.sort()
rn50_scaling_ablation_conditions = list(rn50_scaling_ablation_conditions)
rn50_scaling_ablation_conditions.sort()


In [ ]:
### Color palette

palette = plt.get_cmap('Set1')
condition_to_color.update({c: palette(i / len(rn50_standard_conditions)) for i, c in enumerate(rn50_standard_conditions)})
palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(rn50_fitted_on_OOD_conditions)) for i, c in enumerate(rn50_fitted_on_OOD_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(rn50_ef_conditions)) for i, c in enumerate(rn50_ef_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(rn50_fitted_on_OOD_ef_conditions)) for i, c in enumerate(rn50_fitted_on_OOD_ef_conditions)})

palette = plt.get_cmap('Set2')
condition_to_color.update({c: palette(i / len(rn50_scaling_ablation_conditions)) for i, c in enumerate(rn50_scaling_ablation_conditions)})

condition_to_color["MAP"] = "darkblue"

In [ ]:
wanted_conditions = rn50_all_unique_conditions
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered)

In [ ]:
# TODO
# Concise plots:
# Baseline (MAP, LLLA, TS(pycalib)) (+ TS(mine?))
wanted_conditions = ["MAP", "LLLA", "TS (pycalib)", "TS (WITS)"]
wanted_conditions = ['RN50 ' + c for c in wanted_conditions]
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)

combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, legend_order_permutation=[2, 1, 0, 3])


In [ ]:
# WITS / Laplace with Cov-scaling

wanted_conditions = deepcopy(rn50_standard_conditions)
wanted_conditions.remove("RN50 TS (pycalib)")
conditions_amazon_filtered, results_amazon_filtered = filter_conditions(conditions_amazon, results_amazon, wanted_conditions)
conditions_camelyon17_filtered, results_camelyon17_filtered = filter_conditions(conditions_camelyon17, results_camelyon17, wanted_conditions)
conditions_SkinLesions_filtered, results_SkinLesions_filtered = filter_conditions(conditions_SkinLesions, results_SkinLesions, wanted_conditions)


combined_plot(conditions_amazon_filtered, results_amazon_filtered, conditions_camelyon17_filtered, results_camelyon17_filtered, conditions_SkinLesions_filtered, results_SkinLesions_filtered, legend_order_permutation=[3, 1, 0, 5, 2, 4])


# Amazon

In [ ]:

fig, ax = plt.subplots()

for condition, results in zip(conditions_amazon, results_amazon):
    ax.scatter(range(2), [i['ece'] for i in results], label=condition)

ax.set_xticks([0,1])
ax.set_xticklabels(["ID", "OOD"])

plt.title("ECE on amazon")
plt.legend()
plt.show()



In [ ]:

fig, ax = plt.subplots()

for condition, results in zip(conditions_amazon, results_amazon):
    ax.scatter(range(2), [i['nll'] for i in results], label=condition)

ax.set_xticks([0,1])
ax.set_xticklabels(["ID", "OOD"])

plt.title("NLL on amazon")
plt.legend()
plt.show()



In [ ]:

fig, ax = plt.subplots()

for condition, results in zip(conditions_camelyon17, results_camelyon17):
    ax.scatter(range(2), [i['ece'] for i in results], label=condition)

ax.set_xticks([0,1])
ax.set_xticklabels(["ID", "OOD"])

plt.title("ECE on camelyon17")
plt.legend()
plt.show()



In [ ]:

fig, ax = plt.subplots()

for condition, results in zip(conditions_camelyon17, results_camelyon17):
    ax.scatter(range(2), [i['nll'] for i in results], label=condition)

ax.set_xticks([0,1])
ax.set_xticklabels(["ID", "OOD"])

plt.title("NLL on camelyon17")
plt.legend()
plt.show()



In [ ]:
fig, ax = plt.subplots()


for condition, results in zip(conditions_amazon, results_amazon):
    try:
        acc = [m["acc"] for m in results]
        conf = [m["conf"] for m in results]
        mean_variances = [m["mean_variance"] for m in results]

        ax.plot(range(2), acc, label=f"acc - {condition}")
        ax.errorbar([0, 1], conf, yerr=mean_variances, label=f"conf - {condition}", fmt='-o')
    except: # MAP does not have 'mean_variance'
        acc = [m["acc"] for m in results]
        conf = [m["conf"] for m in results]
        ax.plot(range(2), acc, label=f"acc - {condition}")
        ax.plot(range(2), conf, label=f"conf - {condition}")

ax.set_xticks([0,1])
ax.set_xticklabels(["ID", "OOD"])

plt.title("Accuracy/Confidence on amazon")
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()


for condition, results in zip(conditions_camelyon17, results_camelyon17):
    try:
        acc = [m["acc"] for m in results]
        conf = [m["conf"] for m in results]
        mean_variances = [m["mean_variance"] for m in results]

        ax.plot(range(2), acc, label=f"acc - {condition}")
        ax.errorbar([0, 1], conf, yerr=mean_variances, label=f"conf - {condition}", fmt='-o')
    except: # MAP does not have 'mean_variance'
        acc = [m["acc"] for m in results]
        conf = [m["conf"] for m in results]
        ax.plot(range(2), acc, label=f"acc - {condition}")
        ax.plot(range(2), conf, label=f"conf - {condition}")

ax.set_xticks([0,1])
ax.set_xticklabels(["ID", "OOD"])

plt.title("Accuracy/Confidence on camelyon17")
plt.legend()
plt.show()